# Project Presentation

## Contents

### JuMEG_Installer

- Intention

- Code Examples

- Problems

### File Selection GUI

- Intention 

- Code Examples

- ToDo

- Problems

## JuMEG_Installer

### Intention

- programme to install jumeg with mne via anaconda

- need of merging the mne and the jumeg environment files

- the mne file is preferred at the merge 

### Code Examples

#### Argument Parser

In [1]:
def get_args(argv,parser=None,defaults=None,version=None):
   """
   function to parse the given parameters from the shell
   """
   description="jumeg_installer start parameters"
   h_cuda="Downloads jumeg file with cuda support"
   h_verbose="Makes the installation more verbose"
   if not parser:
      parser = argparse.ArgumentParser(description=description)
   else:
      parser.description=description
   if not defaults:
      defaults={
         "verbose":False,
         }
   if len(argv)<2:
       parser.print_help()
       sys.exit(-1)
   parser.add_argument("-v,--verbose",action="store_true",help=h_verbose,default=defaults.get("verbose"))
   opt=parser.parse_args()
   return opt

#### Loading yaml files

In [3]:
def load_jumeg(opt):
   """
   function to load the jumeg environment file as a python dict
   """
   dict_jumeg=dict()
   if opt.fjumeg:
      fname = opt.fjumeg
   elif opt.cuda:
      subprocess.run(["curl","--remote-name",remote_name.get("jumeg_cuda")])
      fname = os.path.basename( remote_name.get("jumeg_cuda") )
   else:
      subprocess.run(["curl","--remote-name",remote_name.get("jumeg")])
      fname = os.path.basename( remote_name.get("jumeg") )

   with open(fname,"r") as f:
        dict_jumeg = yaml.safe_load(f)
   return dict_jumeg

#### Merging

In [4]:
def update_and_merge(din, u, depth=-1,do_copy=True):
       """ update and merge dict parameter overwrite defaults
       ----------
       dict with defaults parameter
       dict with parameter to merge or update
       depth: recusive level <-1>
       -------
       dict with merged and updated parameters
       """
       if do_copy:
          d = deepcopy(din)
       else:
          d = din 
       for k, v in u.items():
           if isinstance(v, collections.abc.Mapping) and not depth == 0:
              r = update_and_merge(d.get(k, {}), v, depth=max(depth - 1, -1))
              d[k] = r
           elif isinstance(d, collections.abc.Mapping):
              d[k] = u[k]
           else:
              d = {k: u[k]}
       return d 

#### Installation or Updating

In [6]:
def install(opt):
    """
    function to install the new conda environment
    """
    if opt.install:
        fname=opt.name + ".yaml"
        if check_envs(opt):
            subprocess.run(["conda","deactivate"],stdout=DEVNULL)
            subprocess.run(["conda","env","update","-n",opt.name,"--file",fname])
            subprocess.run(["conda","activate",opt.name],stdout=DEVNULL)
        else:
            subprocess.run(["conda","env","create","-f",fname])
            subprocess.run(["conda","activate",opt.name],stdout=DEVNULL)

### Problems

- Working with subprocess outputs

- Installation

## File Selection GUI

### Intention

- programme to select files and config parameters to send it to another programme

- Combination of ListCtrl and a TreeCtrl

- usage of an I/O class

- usage of pubsub for the communication between the Controls

### Code Examples

#### Input method for the ListBox

In [10]:
   def _read_lines(self, f):
      self._file_list=list()
      for line in f:
         line = f.readline().strip()
         if line.startswith("#"): continue
         self._file_list.append(line)
      return self._file_list
         
   def read_file(self,fname):
      self.clear()
      if fname:
         if os.path.exists(fname):
            with open(fname,"r") as f:
                 self._read_lines(f)
      pub.sendMessage(self._name,data=self.get_basenames() ) 
      return self._file_list

#### Base class

In [12]:
import wx
class _BASE(wx.Panel):
   def __init__(self,parent,**kwargs):
      super().__init__(parent,style=wx.BORDER_SUNKEN)
      self.SetBackgroundColour( kwargs.get("bg","grey70") ) 
      self._init(**kwargs)
      self._wx_init(**kwargs)
      self._ApplyLayout()
      self._ApplyFinalLayout()
      self._init_pubsub()
   
   def _init(self,**kwargs):
       pass   
   
   def _wx_init(self,**kwargs):
       pass
      
   def ClickOnCtrl(self,evt):
       evt.Skip()
       
   def _ApplyLayout(self):
       pass

#### init of ListCtrl

In [13]:
   def _wx_init(self,**kwargs):
     
     #-- file selection counter
      self._FSC = wx.StaticText(self, wx.ID_ANY,style=wx.ALIGN_RIGHT)
      self._FSC.SetForegroundColour('red')
     #-- FileListBox
      self._FLB = wx.ListBox(self,wx.ID_ANY,choices=[],style=wx.LB_MULTIPLE) 
      self._FLB.SetFont(wx.Font(12,75,90,90,False,wx.EmptyString))
      self._FLB.Bind(wx.EVT_MOTION,self.OnMouseMove)
      
      self.Bind(wx.EVT_LISTBOX,self.ClickOnSelect) 
      self._update_file_selection_counter()

#### Click on Button

In [14]:
    def ClickOnButton(self,data=None):
        if data.upper().endswith("APPLY"):
           if self._FSB:
              if self._ConfigCtrl._CFG._CfgTreeCtrl:
                 if self._FSB.GetSelectedFiles():
                    files=self._FSB.GetSelectedFiles()
                    self._FSB._FileReader.write_tmp_files(fdata=files,cdata=self._ConfigCtrl._CFG._CfgTreeCtrl.GetData())
                 else:
                    self.ApplyException("You need to select a ListBox item")
              else:
                 self.ApplyException("You need to select a Config file")
                    
        if data.upper().endswith("DE/Select"):
           self._FSB.DeSelectFiles()
        
        if data.upper().endswith("INFO"):
           logger.info(self._FSB.GetSelectedFiles())

#### Generating temporary files

In [15]:
   def write_tmp_files(self,fdata=list(),cdata=dict()):
      CFG=jumeg_base_config.JuMEG_CONFIG()
      user=getpass.getuser()
      count=1
      num=str(count).zfill(3)
      ffname="test_file_"+user+"_"+num+".txt"
      cfname="test_config_"+user+"_"+num+".yaml"
      while os.path.exists(ffname) or os.path.exists(cfname):
         count+=1
         num=str(count).zfill(3)
         ffname="test_file_"+user+"_"+num+".txt"
         cfname="test_config_"+user+"_"+num+".yaml"
      with open(ffname,"w") as f:
         f.writelines("%s\n" % line for line in fdata)
      CFG.save_cfg(fname=cfname,data=cdata)
      print(ffname)
      print(cfname)

### ToDo

- add docstrings

- eliminate dead code and debug code

- testing and some bugfixes

### Problems

- structuring the code

- working with pubsub

- Boxing the controls